# Example queries for geographic locations on COVID-19 Knowledge Graph
[Work in progress]

This notebook demonstrates how to run Cypher queries in a Jupyter Notebook by connecting to a database server.

In [10]:
import datetime
import pandas as pd
from py2neo import Graph

In [11]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [12]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

### List Node Metadata

In [13]:
query = """
MATCH (n:NodeMetadata)
RETURN n.name, n.shortDescription, n.description, n.example, n.definitionSource, n.dataProviders
"""
graph.run(query).to_data_frame()

,n.name,n.shortDescription,n.description,n.example,n.definitionSource,n.dataProviders
0,Location,Geographic location,A geograpic location,"World, ..., Country, State, Country, City, Cru...",None,None
1,World,The World,Top level location,None,None,None
2,UNRegion,Continental regions,Continental regions according to the M49 stan...,Americas,None,None
3,UNSubRegion,Subcontinental regions,Subcontinental regions according to the M49 st...,Latin America and the Caribbean,None,None
4,UNIntermediateRegion,Subdivisions of subcontinental regions,Subdivisions of subcontinental regions accordi...,Caribbean,None,None
5,Country,Countries and dependent Territories,Countries and dependent Territories defined b...,United States,None,None
6,Admin1,"State, Province, Municipality","First administrative divisions, e.g, State, Pr...",California,None,None
7,Admin2,County,Second administrative divisions: County in the US,San Diego County,None,None
8,City,City,City,San Diego,None,None
9,PostalCode,Postal Code,E.g.,a ZIP Code is a postal code used by the Unite...,None,None


### Run a full text search for location names

In [14]:
query = """
CALL db.index.fulltext.queryNodes("locations", "Los Angeles") YIELD node, score
RETURN node.name, node.location, node.population, labels(node), score
"""
df = graph.run(query).to_data_frame()
df.head(25)

,node.name,node.location,node.population,labels(node),score
0,Los Angeles,None,1144.0,"[Location, City]",5.888811
1,Los Angeles,None,4217.0,"[Location, City]",5.888811
2,Los Angeles,None,1616.0,"[Location, City]",5.888811
3,Los Angeles,None,1780.0,"[Location, City]",5.888811
4,Los Angeles,None,250.0,"[Location, City]",5.888811
5,Los Angeles,None,342.0,"[Location, City]",5.888811
6,Los Angeles,None,4054.0,"[Location, City]",5.888811
7,Los Angeles,None,3971883.0,"[Location, City]",5.888811
8,East Los Angeles,None,126496.0,"[Location, City]",4.704535
9,Fraccionamiento los Angeles,None,6282.0,"[Location, City]",4.704535


### Calculate distances between locations

In [15]:
query = """
MATCH (a:City {name:"Denver"}), (b: City {name:"Houston"})
WHERE a.elevation > 1000 AND b.elevation < 25
RETURN a.name, a.elevation, a.location, b.name, b.elevation, b.location, distance(a.location, b.location) / 1000.0 as distance_KM
"""
graph.run(query).to_data_frame()

,a.name,a.elevation,a.location,b.name,b.elevation,b.location,distance_KM
0,Denver,1609,None,Houston,12,None,None


### Find nearby locations
Distances are in meters

In [16]:
# Example from: https://maxdemarzi.com/2018/03/21/neo4j-geospatial-queries/
query = """
MATCH (c:City)-[:IN*]->(s:Admin1), (c2:City)
WHERE c.name = "Union City"
  AND s.name = "California"
  AND distance(c.location, c2.location) <= 10000
RETURN c2.name, c2.location.longitude, c2.location.latitude, distance(c.location, c2.location)
"""
graph.run(query).to_data_frame()

""
